In [4]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder


In [5]:
df = pd.read_csv('/content/covid_toy.csv')
df


,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No
...,...,...,...,...,...,...
95,12,Female,104.0,Mild,Bangalore,No
96,51,Female,101.0,Strong,Kolkata,Yes
97,20,Female,101.0,Mild,Bangalore,No
98,5,Female,98.0,Strong,Mumbai,No


In [6]:
df['city'].value_counts()

,count
city,
Kolkata,32
Bangalore,30
Delhi,22
Mumbai,16


In [7]:
df.isnull().sum()

,0
age,0
gender,0
fever,10
cough,0
city,0
has_covid,0


In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2)
X_train

,age,gender,fever,cough,city
32,34,Female,101.0,Strong,Delhi
16,69,Female,103.0,Mild,Kolkata
83,17,Female,104.0,Mild,Kolkata
50,19,Male,101.0,Mild,Delhi
33,26,Female,98.0,Mild,Kolkata
...,...,...,...,...,...
27,33,Female,102.0,Strong,Delhi
51,11,Female,100.0,Strong,Kolkata
18,64,Female,98.0,Mild,Bangalore
25,23,Male,NaN,Mild,Mumbai


In [9]:
X_test

,age,gender,fever,cough,city
73,34,Male,98.0,Strong,Kolkata
23,80,Female,98.0,Mild,Delhi
59,6,Female,104.0,Mild,Kolkata
13,64,Male,102.0,Mild,Bangalore
2,42,Male,101.0,Mild,Delhi
99,10,Female,98.0,Strong,Kolkata
94,79,Male,NaN,Strong,Kolkata
87,47,Male,101.0,Strong,Bangalore
26,19,Female,100.0,Mild,Kolkata
40,49,Female,102.0,Mild,Delhi


In [10]:
X_test.shape

(20, 5)

In [11]:
X_train.shape

(80, 5)

In [12]:
#adding simple imputer to fever column
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

#also the test data
X_test_fever = si.fit_transform(X_test[['fever']])
X_train_fever.shape

(80, 1)

In [13]:
#OrdinalEncoding -> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

#also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])
X_train_cough.shape

(80, 1)

In [16]:
#OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first', sparse_output=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

#also the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])
X_test_gender_city.shape

(20, 4)

In [17]:
#Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

#also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values
X_train_age.shape

(80, 1)

In [18]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
#alse the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

# **Direct column Transfer Another way**

In [20]:
from sklearn.compose import ColumnTransformer
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
('tnf3',OneHotEncoder(drop='first',sparse_output=False),['gender','city'])
],remainder='passthrough')

transformer.fit_transform(X_train).shape
transformer.transform(X_test).shape

(20, 7)